# 基于机器学习数据库飞速上线AI应用
大家平时可能都会打车，从出发的地点到目的地，行程耗时可能会存在多种因素，比如天气，是否周五，如果获取更准确的耗时预测，对人来说是一个复杂的问题，而对机器就变得很简单，今天的任务就是开发一个通过机器学习模型进行出租车行程耗时的实时智能应用，整个应用开发是基于[notebook](http://ipython.org/notebook.html)



![出租车行程耗时预测](https://th.bing.com/th/id/Rcf52e9678006c3e99a98cf88a216e38d?rik=oQN4iVqyXXjYNg&riu=http%3a%2f%2fi1.hexun.com%2f2020-05-08%2f201272779.jpg&ehk=4eiCf7x4YL8wYL4D6wnqUybMtVEER6teKg1deOEcYys%3d&risl=&pid=ImgRaw)

## 初始化环境
整个初始化过程包含安装fedb，以及相关运行环境，初始化脚步可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/init.sh

In [ ]:
!cd demo && sh init.sh

## 导入行程历史数据到fedb

使用fedb进行时序特征计算是需要历史数据的，所以我们将历史的行程数据导入到fedb，以便实时推理可以使用历史数据进行特征推理，导入代码可以参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/import.py

In [ ]:
!cd demo && python3 import.py

## 使用行程数据进行模型训练

通过label数据进行模型训练，一下是这次任务使用的代码

* 训练脚本代码 https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/train_sql.py 
* 训练数据 https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/data/taxi_tour_table_train_simple.snappy.parquet

整个任务最终会生成一个model.txt

In [ ]:
!cd demo && sh train.sh

## 使用训练的模型搭建链接fedb的实时推理http服务

基于上一步生成的模型和fedb历史数据，搭建一个实时推理服务，整个推理服务代码参考https://github.com/4paradigm/DemoApps/blob/main/predict-taxi-trip-duration-nb/demo/predict_server.py

In [ ]:
!cd demo && sh start_predict_server.sh

## 通过http请求发送一个推理请求

整个请求很简单，具体代码如下

```python
url = "http://127.0.0.1:8887/predict"
req ={"id":"id0376262",
	"vendor_id":1,
	"pickup_datetime":1467302350000,
	"dropoff_datetime":1467304896000,
	"passenger_count":2,
	"pickup_longitude":-73.873093,
	"pickup_latitude":40.774097,
	"dropoff_longitude":-73.926704,
	"dropoff_latitude":40.856739,
	"store_and_fwd_flag":"N",
	"trip_duration":1}
r = requests.post(url, json=req)
print(r.text)
print("Congraduation! You have finished the task.")
tmp = os.urandom(44)
secret_key = base64.b64encode(tmp)
print("Your Key:" + str(secret_key))
```

In [ ]:
!cd demo && python3 predict.py